for brief matrix explanations read-> https://github.com/USEPA/USEEIO_API/blob/master/doc/data_format.md
# Note:
If rounding off 8 decimals, ozone depletion, pesticides and a few others would need to switched to scientific notation in the data file. This would allow the files to be reduced
US from 151kb to under 72.7kb
GA from 120kb, to under 59.2kb

In [1]:
#BASE URL AND KEY
#Use the requests library
import csv
import zipfile, io
import os
import pathlib
import requests as r
import pandas as pd
pd.options.display.max_rows = 999

#base_url ='https://api.edap-cluster.com/useeio/api'
base_url ='https://smmtool.app.cloud.gov/api'

api_headers = {}
# api-key for USEEIO
#with open(".USEEIO_API_KEY", 'r') as KEY_FILE:
#    api_headers['x-api-key']=KEY_FILE.read()
#api_headers['x-api-key']="PUT_API_KEY_HERE"

In [2]:
# AVAILABLE MODELS
# Show the available models in the native JSON output
models = r.get(base_url+'/models',headers=api_headers)
models_json = models.json()

# MODEL NAME
#Use the first available model
model_name = models_json[0]['id']

#MODEL SECTORS
#Get the sectors
url = base_url+'/'+model_name+'/sectors'
model_sectors = r.get(url,headers=api_headers)
model_sectors_json = model_sectors.json()
#print(model_sectors_json)
model_sector_ids = []
for i in model_sectors_json:
    model_sector_ids.append(i['id'])
first_sector_id = model_sector_ids[0]
first_sector_name = model_sectors_json[0]['name']
print('The '+ model_name +' model has ' + str(len(model_sectors_json)) + ' sectors.')

# INDICATORS
url = base_url+'/'+model_name+'/indicators'
model_indictrs_response = r.get(url,headers=api_headers)
model_indictrs_json = model_indictrs_response.json()
HC_indictr_id = model_indictrs_json[8]['id']
model_indictrs_names = []
model_indictrs_ids = []
for i in model_indictrs_json:
    model_indictrs_names.append(i['name'])
    model_indictrs_ids.append(i['id'])
print('The '+ model_name +' model has ' + str(len(model_indictrs_json)) + ' indicators. The names are:')
print(model_indictrs_names)

The USEEIOv2.0 model has 411 sectors.
The USEEIOv2.0 model has 23 indicators. The names are:
['Acidification Potential', 'Commercial Construction and Demolition Debris', 'Commercial Municipal Solid Waste', 'Commercial RCRA Hazardous Waste', 'Energy Use', 'Eutrophication Potential', 'Freshwater Ecotoxicity Potential', 'Freshwater withdrawals', 'Greenhouse Gases', 'Hazardous Air Pollutants', 'Human Health - Cancer', 'Human Health - Noncancer', 'Human Health - Respiratory Effects', 'Human Health Toxicity', 'Jobs Supported', 'Land use', 'Minerals and Metals Use', 'Nonrenewable Energy Use', 'Ozone Depletion', 'Pesticides', 'Renewable Energy Use', 'Smog Formation Potential', 'Value Added']


In [3]:
model_indictrs_ids

['Impact Potential/ACID/kg SO2 eq',
 'Waste Generated/CCDD/kg',
 'Waste Generated/CMSW/kg',
 'Waste Generated/CRHW/kg',
 'Resource Use/ENRG/MJ',
 'Impact Potential/EUTR/kg N eq',
 'Impact Potential/ETOX/CTUe',
 'Resource Use/WATR/kg',
 'Impact Potential/GHG/kg CO2 eq',
 'Chemical Releases/HAPS/kg',
 'Impact Potential/HCAN/CTUh',
 'Impact Potential/HNCN/CTUh',
 'Impact Potential/HRSP/kg PM2.5 eq',
 'Impact Potential/HTOX/CTUh',
 'Economic & Social/JOBS/jobs',
 'Resource Use/LAND/m2*yr',
 'Resource Use/MNRL/kg',
 'Resource Use/NNRG/MJ',
 'Impact Potential/OZON/kg CFC-11 eq',
 'Chemical Releases/PEST/kg',
 'Resource Use/RNRG/MJ',
 'Impact Potential/SMOG/kg O3 eq',
 'Economic & Social/VADD/$']

In [4]:
#INDICATORS
model_indictrs_response = r.get(base_url+model_name+'/indicators',headers=api_headers)
model_indictrs_json = model_indictrs_response.json()
model_indictrs_json


JSONDecodeError: Extra data: line 1 column 5 (char 4)

In [5]:
#In order to get the impacts of total US production, we need to first get the US_production demand vector to use in the calculation

#First get the name by showing all model demand vectors
#DEMAND VECTORS
#See the demand vectors available for the model in their JSON format
demands_response = r.get(base_url+'/'+model_name+'/demands',headers=api_headers)
demands = demands_response.json()
demands

[{'id': '2012_US_Consumption_Complete',
  'year': 2012,
  'type': 'Consumption',
  'system': 'Complete',
  'location': 'US'},
 {'id': '2012_US_Production_Complete',
  'year': 2012,
  'type': 'Production',
  'system': 'Complete',
  'location': 'US'},
 {'id': '2012_US_Consumption_Household',
  'year': 2012,
  'type': 'Consumption',
  'system': 'Household',
  'location': 'US'},
 {'id': '2012_US_Consumption_Domestic',
  'year': 2012,
  'type': 'Consumption',
  'system': 'Domestic',
  'location': 'US'}]

In [6]:
#url = base_url+'/'+model_name+'/demands/2007_us_production'

url = base_url+'/'+model_name+'/demands/2012_US_Production_Complete'
us_production_response = r.get(url,headers=api_headers)
us_production = us_production_response.json()

In [7]:
# A model calculation also requires defining a perspective which is either direct or final
# Direct perspective associated impacts in the sectors in which they occur
# Final perspective rolls up impacts into those products consumed by final consumers

data_to_post = {"perspective": "direct"}
data_to_post["demand"] = us_production
url = base_url+'/'+model_name+'/calculate'
result_response = r.post(url,headers=api_headers,json=data_to_post)
result = result_response.json()


##Old code = uses D matrix
# D0_response = r.get(base_url+model_name+'/matrix/D?row=0',headers=api_headers)
# D0 = D0_response.json()
# sorted(D0)

#D0_response = r.get(base_url+model_name+'/matrix/D',headers=api_headers)
#D0 = D0_response.json()

# sorted(D0[0])

In [8]:
#
# Collect ALL Indicators from ALL Industr Sectors and Limit to Georgia
#

# D0_response = r.get(base_url+model_name+'/matrix/D?row=0',headers=api_headers)
#D0_response = r.get(base_url+model_name+'/matrix/D',headers=api_headers)
#D0 = D0_response.json()
#print(D0)
#indicator_data = {}
# Add the industry sector IDs
# indicator_data.update({'industry':model_sector_ids})
#indicator_data.update({'industry_code':[_.split('/')[0] for _ in model_sector_ids]})
#indicator_data.update({'industry_detail':[_.split('/')[1] for _ in model_sector_ids]})
#indicator_data.update({'industry_region':[_.split('/')[2] for _ in model_sector_ids]})

sectors_df = pd.DataFrame(model_sectors_json)

result_df = pd.DataFrame(data=result['data'],columns=result['sectors'],index=result['indicators']).transpose()
#for key, values_list in zip(model_indictrs_ids, result):
#    indicator_data.update({key:values_list})

#all_indic_all_indust = pd.DataFrame(data=indicator_data)
# all_indic_all_indust.loc[0:20,['industry_code', 'industry_detail', 'ACID']].sort_values(by='ACID', ascending=False)

formatted_result = pd.merge(sectors_df,result_df,left_on='id',right_index=True)

formatted_result = formatted_result.drop(columns=["id","index","description"])

formatted_result = formatted_result.rename(columns={"code":"industry_code","name":"industry_detail","location":"industry_region"})

formatted_result.head(50)

,industry_detail,industry_code,industry_region,Impact Potential/ACID/kg SO2 eq,Waste Generated/CCDD/kg,Waste Generated/CMSW/kg,Waste Generated/CRHW/kg,Resource Use/ENRG/MJ,Impact Potential/EUTR/kg N eq,Impact Potential/ETOX/CTUe,...,Impact Potential/HTOX/CTUh,Economic & Social/JOBS/jobs,Resource Use/LAND/m2*yr,Resource Use/MNRL/kg,Resource Use/NNRG/MJ,Impact Potential/OZON/kg CFC-11 eq,Chemical Releases/PEST/kg,Resource Use/RNRG/MJ,Impact Potential/SMOG/kg O3 eq,Economic & Social/VADD/$
0,"Fresh soybeans, canola, flaxseeds, and other o...",1111A0,US,1.689989e+08,6.313557e+06,9.235378e+07,1.209144e+05,0.000000e+00,4.524585e+08,1.598500e+12,...,150.539387,4128.291408,4.191327e+11,0.000000e+00,0.000000e+00,0.000000,2.186474e+07,0.000000e+00,6.053902e+08,2.605244e+10
1,"Fresh wheat, corn, rice, and other grains",1111B0,US,1.237849e+09,1.191217e+07,1.742495e+08,1.464490e+05,0.000000e+00,2.751483e+09,9.718492e+11,...,250.782185,49899.408361,8.414299e+11,0.000000e+00,0.000000e+00,0.000000,5.683273e+07,0.000000e+00,2.431745e+09,2.527112e+10
2,"Fresh vegetables, melons, and potatoes",111200,US,9.033595e+07,3.655473e+06,5.347173e+07,7.326338e+04,6.667892e+07,1.504704e+08,5.368857e+11,...,60.071799,99521.675472,2.205783e+10,0.000000e+00,0.000000e+00,186480.112201,2.499440e+07,6.667892e+07,2.197485e+08,8.072573e+09
3,Fresh fruits and tree nuts,111300,US,1.029882e+08,8.261633e+06,1.208500e+08,1.497482e+05,0.000000e+00,7.474252e+06,7.743873e+11,...,58.388842,228421.457463,3.613913e+10,0.000000e+00,0.000000e+00,56692.189280,1.977082e+07,0.000000e+00,3.152700e+08,2.346966e+10
4,"Greenhouse crops, mushrooms, nurseries, and fl...",111400,US,5.152436e+07,2.636343e+06,3.664794e+07,2.853950e+04,1.123735e+05,3.247503e+06,1.450595e+07,...,29.268148,99269.920876,4.972849e+09,0.000000e+00,0.000000e+00,25.093775,0.000000e+00,1.123735e+05,2.844713e+08,1.058448e+10
5,"Tobacco, cotton, sugarcane, peanuts, sugar bee...",111900,US,6.834781e+07,2.680677e+06,3.921255e+07,6.264071e+04,1.132745e+08,7.820786e+07,4.205699e+11,...,106.506871,33036.403714,4.087800e+11,0.000000e+00,0.000000e+00,0.000000,5.978131e+06,1.132745e+08,2.635707e+08,9.837003e+09
6,Dairies,112120,US,7.614672e+08,2.846209e+06,4.163393e+07,2.375317e+04,0.000000e+00,6.476255e+07,3.276212e+07,...,44.729318,96657.266960,9.091125e+10,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,6.332931e+08,7.388826e+09
7,Cattle ranches and feedlots,1121A0,US,1.264565e+09,7.373879e+06,1.078640e+08,6.153904e+04,0.000000e+00,1.173516e+08,5.350693e+07,...,97.865370,52717.049154,2.608257e+12,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.349491e+09,2.537045e+10
8,Poultry farms,112300,US,6.640485e+08,3.577799e+06,5.233552e+07,3.267968e+04,0.000000e+00,4.274701e+07,2.059390e+07,...,37.751744,36368.687718,2.777679e+10,0.000000e+00,0.000000e+00,0.149644,0.000000e+00,0.000000e+00,4.487880e+08,3.697495e+09
9,Animal farms and aquaculture ponds (except cat...,112A00,US,1.934448e+09,3.806114e+06,5.543430e+07,9.766049e+04,4.850349e+05,1.295672e+08,1.811050e+09,...,26.003785,79365.917503,7.307458e+11,0.000000e+00,0.000000e+00,0.315555,0.000000e+00,4.850349e+05,6.021379e+08,2.837911e+10


In [9]:
#all_indic_all_indust.min()

NameError: name 'all_indic_all_indust' is not defined

In [89]:
#all_indic_all_indust=all_indic_all_indust[all_indic_all_indust['ENRG'] !=0]

In [91]:
#all_indic_all_indust=all_indic_all_indust.replace(0,0.0000000000000000000000001)

In [92]:
#all_indic_all_indust.min()

industry_code                 111200
industry_detail    abrasive products
industry_region                   us
ACID                     7.11381e-08
ENRG                     4.09471e-09
ETOX                     2.00838e-08
EUTR                    -2.55231e-08
FOOD                           1e-25
GCC                      0.000212623
HAPS                     1.02198e-08
HAZW                           1e-25
HC                       1.10792e-14
HNC                      2.15618e-14
HRSP                     9.28547e-09
HTOX                      3.2663e-14
JOBS                      5.0741e-09
LAND                     1.77919e-05
METL                    -1.08526e-10
MINE                           1e-25
MSW                            1e-25
NREN                           1e-25
OZON                           1e-25
PEST                    -2.27252e-14
REN                            1e-25
SMOG                     2.99038e-06
VADD                       0.0617671
WATR                     1.38524e-05
d

In [93]:
#all_indic_all_indust.max()

industry_code                                  s00700
industry_detail    wooden windows, door, and flooring
industry_region                                    us
ACID                                        0.0310546
ENRG                                          372.441
ETOX                                          28.3965
EUTR                                        0.0465659
FOOD                                         0.023746
GCC                                            6.1599
HAPS                                       0.00515986
HAZW                                         0.264547
HC                                        2.39431e-09
HNC                                       1.66197e-08
HRSP                                        0.0160841
HTOX                                      1.90192e-08
JOBS                                      0.000186337
LAND                                          91.3194
METL                                      1.30526e-05
MINE                        

In [10]:
#all_indic_all_indust.to_csv('indicators_sectors.csv')

formatted_result.to_csv('indicators_sectors.csv')

# GAUSEEIO

In [12]:
# AVAILABLE MODELS
# Show the available models in the native JSON output
models = r.get(base_url+'/models',headers=api_headers)
models_json = models.json()

# MODEL NAME
#Use the first available model
model_name = models_json[1]['id']

#MODEL SECTORS
#Get the sectors
url = base_url+'/'+model_name+'/sectors'
model_sectors = r.get(base_url+'/'+model_name+'/sectors',headers=api_headers)

model_sectors_json = model_sectors.json()
#print(model_sectors_json)
model_sector_ids = []
for i in model_sectors_json:
    model_sector_ids.append(i['id'])
first_sector_id = model_sector_ids[0]
first_sector_name = model_sectors_json[0]['name']
print('The '+ model_name +' model has ' + str(len(model_sectors_json)) + ' sectors.')

# INDICATORS
model_indictrs_response = r.get(base_url+'/'+model_name+'/indicators',headers=api_headers)
model_indictrs_json = model_indictrs_response.json()
HC_indictr_id = model_indictrs_json[8]['id']
model_indictrs_names = []
model_indictrs_ids = []
for i in model_indictrs_json:
    model_indictrs_names.append(i['name'])
    model_indictrs_ids.append(i['id'])
print('The '+ model_name +' model has ' + str(len(model_indictrs_json)) + ' indicators. The names are:')
print(model_indictrs_names)

IndexError: list index out of range

In [3]:
model_indictrs_ids

['ACID',
 'ENRG',
 'ETOX',
 'EUTR',
 'FOOD',
 'GCC',
 'HAPS',
 'HAZW',
 'HC',
 'HNC',
 'HRSP',
 'HTOX',
 'JOBS',
 'LAND',
 'METL',
 'MINE',
 'MSW',
 'NREN',
 'OZON',
 'PEST',
 'REN',
 'SMOG',
 'VADD',
 'WATR']

In [4]:
#INDICATORS
model_indictrs_response = r.get(base_url+model_name+'/indicators',headers=api_headers)
model_indictrs_json = model_indictrs_response.json()
model_indictrs_json

[{'id': 'ACID',
  'index': 0,
  'name': 'Acid Rain',
  'code': 'ACID',
  'unit': 'kg SO2 eq',
  'group': 'Impact Potential'},
 {'id': 'ENRG',
  'index': 1,
  'name': 'Energy',
  'code': 'ENRG',
  'unit': 'MJ',
  'group': 'Resource Use'},
 {'id': 'ETOX',
  'index': 2,
  'name': 'Freshwater Aquatic Ecotoxicity',
  'code': 'ETOX',
  'unit': 'CTUe',
  'group': 'Impact Potential'},
 {'id': 'EUTR',
  'index': 3,
  'name': 'Eutrophication',
  'code': 'EUTR',
  'unit': 'kg N eq',
  'group': 'Impact Potential'},
 {'id': 'FOOD',
  'index': 4,
  'name': 'Food Waste',
  'code': 'FOOD',
  'unit': 'kg',
  'group': 'Waste Generated'},
 {'id': 'GCC',
  'index': 5,
  'name': 'Global Climate Change',
  'code': 'GCC',
  'unit': 'kg CO2 eq',
  'group': 'Impact Potential'},
 {'id': 'HAPS',
  'index': 6,
  'name': 'Hazardous Air Pollutants',
  'code': 'HAPS',
  'unit': 'kg',
  'group': 'Chemical Releases'},
 {'id': 'HAZW',
  'index': 7,
  'name': 'Hazardous Waste',
  'code': 'HAZW',
  'unit': 'kg',
  'group

In [5]:
# D0_response = r.get(base_url+model_name+'/matrix/D?row=0',headers=api_headers)
# D0 = D0_response.json()
# sorted(D0)

D0_response = r.get(base_url+model_name+'/matrix/D',headers=api_headers)
D0 = D0_response.json()

# sorted(D0[0])

In [6]:
#
# Collect ALL Indicators from ALL Industr Sectors and Limit to Georgia
#

# D0_response = r.get(base_url+model_name+'/matrix/D?row=0',headers=api_headers)
D0_response = r.get(base_url+model_name+'/matrix/D',headers=api_headers)
D0 = D0_response.json()
#print(D0)
indicator_data = {}
# Add the industry sector IDs
# indicator_data.update({'industry':model_sector_ids})
indicator_data.update({'industry_code':[_.split('/')[0] for _ in model_sector_ids]})
indicator_data.update({'industry_detail':[_.split('/')[1] for _ in model_sector_ids]})
indicator_data.update({'industry_region':[_.split('/')[2] for _ in model_sector_ids]})


for key, values_list in zip(model_indictrs_ids, D0):
    indicator_data.update({key:values_list})

all_indic_all_indust = pd.DataFrame(data=indicator_data)
# all_indic_all_indust.loc[0:20,['industry_code', 'industry_detail', 'ACID']].sort_values(by='ACID', ascending=False)

all_indic_all_indust.head(50)

,industry_code,industry_detail,industry_region,ACID,ENRG,ETOX,EUTR,FOOD,GCC,HAPS,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
0,1111a0,"fresh soybeans, canola, flaxseeds, and other o...",us-ga,0.006991,0.000000e+00,27.141609,0.008084,0.000021,0.549898,9.931177e-05,...,5.481034e-09,0.000000,0.000083,0.000000,5.229164e-06,3.798786e-04,0.000000e+00,0.048809,0.471064,0.009608
1,1111b0,"fresh wheat, corn, rice, and other grains",us-ga,0.023198,0.000000e+00,9.711579,0.033350,0.000130,1.703382,1.312246e-04,...,8.166104e-07,0.000000,0.000506,0.000000,5.712925e-06,5.697599e-04,0.000000e+00,0.059359,-0.002548,0.461626
2,111200,"fresh vegetables, melons, and potatoes",us-ga,0.005637,2.339565e-03,22.179052,0.005695,0.001000,0.354328,3.491954e-04,...,2.390340e-06,0.000000,0.003883,0.000000,1.241849e-05,9.512581e-04,2.339565e-03,0.017317,0.604597,0.451799
3,111300,fresh fruits and tree nuts,us-ga,0.003150,0.000000e+00,13.995239,0.000306,0.001330,0.235407,9.095916e-05,...,1.030074e-05,0.000000,0.005168,0.000000,6.172102e-06,2.091238e-04,0.000000e+00,0.016048,0.605345,0.403344
4,111400,"greenhouse crops, mushrooms, nurseries, and fl...",us-ga,0.000780,3.631625e-06,0.006251,0.000097,0.001812,0.657274,9.637975e-05,...,1.858372e-08,0.000000,0.007078,0.000000,5.236234e-06,4.605271e-05,3.631625e-06,0.032099,0.534020,0.056251
5,111900,"tobacco, cotton, sugarcane, peanuts, sugar bee...",us-ga,0.004418,4.387845e-03,24.673328,0.009326,0.000498,0.907799,2.411416e-04,...,5.715402e-09,0.000000,0.001935,0.000000,5.243500e-06,5.245319e-04,4.387845e-03,0.035231,0.466042,2.725868
6,112120,dairies,us-ga,0.015980,0.000000e+00,0.000341,0.001451,0.000709,1.723280,2.664576e-05,...,4.298771e-09,0.000000,0.002753,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.026734,0.325316,0.029922
7,1121a0,cattle ranches and feedlots,us-ga,0.015492,0.000000e+00,0.000434,0.001429,0.000148,3.366082,3.386558e-05,...,6.072406e-09,0.000000,0.000574,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.034565,0.295837,0.070167
8,112300,poultry farms,us-ga,0.020613,0.000000e+00,0.000038,0.001344,0.000150,0.299008,2.971036e-06,...,5.777589e-10,0.000000,0.000581,0.000000,2.935473e-14,0.000000e+00,0.000000e+00,0.002922,0.231893,0.007003
9,112a00,animal farms and aquaculture ponds (except cat...,us-ga,0.023216,1.093109e-05,0.000113,0.001697,0.000380,0.725200,8.892183e-06,...,2.339780e-08,0.000000,0.001485,0.000000,4.383810e-12,1.990412e-12,1.093109e-05,0.008871,0.624700,0.327317


In [7]:
all_indic_all_indust.min()

industry_code                   1111a0
industry_detail      abrasive products
industry_region    gas field machinery
ACID                                 0
ENRG                                 0
ETOX                                 0
EUTR                      -2.57513e-08
FOOD                                 0
GCC                           -40.1141
HAPS                                 0
HAZW                                 0
HC                                   0
HNC                                  0
HRSP                                 0
HTOX                                 0
JOBS                                 0
LAND                                 0
METL                      -1.08701e-10
MINE                                 0
MSW                                  0
NREN                                 0
OZON                                 0
PEST                      -2.28667e-14
REN                                  0
SMOG                                 0
VADD                     

In [8]:
all_indic_all_indust=all_indic_all_indust[all_indic_all_indust['ENRG'] !=0]

In [9]:
all_indic_all_indust=all_indic_all_indust.replace(0,0.0000000000000000000000001)

In [10]:
all_indic_all_indust.min()

industry_code                 111200
industry_detail    abrasive products
industry_region                 rous
ACID                     7.13354e-08
ENRG                      4.1974e-09
ETOX                     2.01385e-08
EUTR                    -2.57513e-08
FOOD                           1e-25
GCC                         -40.1141
HAPS                     1.02354e-08
HAZW                           1e-25
HC                       1.10977e-14
HNC                      2.16114e-14
HRSP                       9.302e-09
HTOX                     3.27311e-14
JOBS                        0.777273
LAND                     1.77776e-05
METL                    -1.08701e-10
MINE                           1e-25
MSW                            1e-25
NREN                           1e-25
OZON                           1e-25
PEST                    -2.28667e-14
REN                            1e-25
SMOG                     2.99462e-06
VADD                       0.0617671
WATR                     1.39408e-05
d

In [11]:
all_indic_all_indust.max()

industry_code                                  s00700
industry_detail    wooden windows, door, and flooring
industry_region                                 us-ga
ACID                                        0.0304141
ENRG                                          372.441
ETOX                                          24.6733
EUTR                                        0.0470181
FOOD                                        0.0237519
GCC                                           6.83454
HAPS                                       0.00494802
HAZW                                         0.179007
HC                                        4.57658e-09
HNC                                       1.68995e-08
HRSP                                        0.0156598
HTOX                                      1.92482e-08
JOBS                                          8056.71
LAND                                          183.933
METL                                      9.43538e-05
MINE                        

In [12]:
all_indic_all_indust=all_indic_all_indust[all_indic_all_indust['industry_region'] =='us-ga']

In [79]:
all_indic_all_indust

,industry_code,industry_detail,industry_region,ACID,ENRG,ETOX,EUTR,FOOD,GCC,HAPS,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
2,111200,"fresh vegetables, melons, and potatoes",us-ga,5.637312e-03,2.339565e-03,2.217905e+01,5.695208e-03,0.001000,0.354328,3.491954e-04,...,2.390340e-06,1.000000e-25,0.003883,1.000000e-25,1.241849e-05,9.512581e-04,2.339565e-03,0.017317,0.604597,0.451799
4,111400,"greenhouse crops, mushrooms, nurseries, and fl...",us-ga,7.797552e-04,3.631625e-06,6.250946e-03,9.668709e-05,0.001812,0.657274,9.637975e-05,...,1.858372e-08,1.000000e-25,0.007078,1.000000e-25,5.236234e-06,4.605271e-05,3.631625e-06,0.032099,0.534020,0.056251
5,111900,"tobacco, cotton, sugarcane, peanuts, sugar bee...",us-ga,4.418486e-03,4.387845e-03,2.467333e+01,9.325671e-03,0.000498,0.907799,2.411416e-04,...,5.715402e-09,1.000000e-25,0.001935,1.000000e-25,5.243500e-06,5.245319e-04,4.387845e-03,0.035231,0.466042,2.725868
9,112a00,animal farms and aquaculture ponds (except cat...,us-ga,2.321560e-02,1.093109e-05,1.128994e-04,1.696674e-03,0.000380,0.725200,8.892183e-06,...,2.339780e-08,1.000000e-25,0.001485,1.000000e-25,4.383810e-12,1.990412e-12,1.093109e-05,0.008871,0.624700,0.327317
10,113000,timber and raw forest products,us-ga,8.750041e-04,1.638421e-04,9.533648e-01,4.536737e-04,0.001346,-25.096940,1.577247e-04,...,7.748314e-08,1.000000e-25,0.005256,1.000000e-25,1.584223e-07,1.948138e-05,1.638421e-04,0.025507,0.594259,0.085341
11,114000,wild-caught fish and game,us-ga,1.420058e-06,1.063777e-05,5.742249e-03,1.413841e-05,0.000533,0.189834,3.411390e-07,...,5.115071e-08,1.000000e-25,0.002086,1.000000e-25,3.181110e-10,1.061927e-07,1.063777e-05,0.000044,0.750106,0.000494
13,211000,unrefined oil and gas,us-ga,2.174912e-03,1.162591e+02,1.712053e-03,3.605606e-04,0.000131,0.445230,3.308863e-04,...,1.366576e-08,1.000000e-25,0.000510,1.162591e+02,1.834051e-08,3.879289e-12,4.932235e-06,0.097348,0.806688,0.003283
14,212100,coal,us-ga,3.735030e-04,3.724406e+02,9.396857e-06,4.513515e-05,0.000331,1.449957,1.028723e-06,...,3.203183e-07,1.000000e-25,0.001285,3.724406e+02,1.524454e-12,1.000000e-25,1.000000e-25,0.012967,0.518176,0.000292
15,212230,"copper, nickel, lead, and zinc",us-ga,4.603268e-04,8.182560e-05,4.876107e-04,2.408367e-05,0.000114,0.240308,5.954111e-06,...,1.582038e-06,6.903255e-02,0.000442,1.000000e-25,2.898588e-10,1.000000e-25,8.182560e-05,0.005616,0.200886,0.006828
16,2122a0,"iron, gold, silver, and other metal ores",us-ga,4.881191e-04,1.464220e-03,7.707477e-04,7.619341e-05,0.000104,0.139169,1.131028e-05,...,6.621019e-07,2.564071e-01,0.000404,1.000000e-25,5.187072e-09,1.000000e-25,1.464220e-03,0.011672,0.176459,0.007318


In [80]:
all_indic_all_indust.to_csv('indicators_sectors_GA.csv')